In [ ]:
def show_critical_semantic_area(locationMap,img_orig):
    locationMap = np.maximum(locationMap, 0)  # ReLU
    locationMap=locationMap-locationMap.min()
    locationMap = locationMap / locationMap.max()

    plt.figure()
    #img_show=cv2.cvtColor(img_orig,cv2.COLOR_BGR2RGB)
    plt.subplot(131)
    plt.imshow(img_orig)
    plt.axis('off')
    plt.tight_layout()

    heatmap = cv2.applyColorMap(np.uint8(255 * locationMap), cv2.COLORMAP_JET) 
    heatmap_show=cv2.cvtColor(heatmap,cv2.COLOR_BGR2RGB)
    plt.subplot(132)
    plt.imshow(heatmap_show)
    plt.axis('off')
    plt.tight_layout()

    semantic_show =0.7* heatmap_show +  img_orig
    semantic_show=np.clip((semantic_show/np.max(semantic_show)),0,1)
    plt.subplot(133)
    plt.imshow(semantic_show)
    plt.axis('off')
    plt.tight_layout()
    
    #plt.savefig('tar_'+savefile+'.jpg')
    plt.show()

In [ ]:
def Grad_CAM_Plus_Plus(img_numpy,img_tensor,target_label,model):
    final_conv_name = None
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            final_conv_name = name
   
    model_module_last_conv=None
    for name,module in model.named_modules():
        if name==final_conv_name:
            model_module_last_conv=module
    
    feature_maps=[]
    def hook_features(module,input,output): #forward hook feature
        feature_maps.append(output)       
    feature_gradients=[]
    def hook_gradient(module, grad_input, grad_output):  #backward hook gradient
        feature_gradients.append(grad_output[0].detach())

    
    hook1=model_module_last_conv.register_forward_hook(hook_features)#register forward hook
    hook2=model_module_last_conv.register_backward_hook(hook_gradient)#register backward hook
   
    logits=model(img_tensor)
    preds=Func.softmax(logits,dim=1)
    probs,idx=preds.data.squeeze().sort(0,True)
    probs=probs.data.cpu().numpy()
    idx=idx.data.cpu().numpy()
    i=np.where(idx==target_label)[0]
   
    model.zero_grad() 
    targ_class_pred= logits[0][idx[i][0]] #logits.shape=(1,1000)
    targ_class_pred.backward(retain_graph=True) #backward pass for gradient
    
    feature_map = feature_maps[0].cpu().data.numpy().squeeze()
    feature_gradient = feature_gradients[0].cpu().data.numpy().squeeze()
    feature_gradient=np.maximum(feature_gradient,0)
    
    alpha_num=feature_gradient**2
    alpha_den=2*(feature_gradient**2)+(feature_gradient**3)*(np.sum(feature_map, axis=(1,2))[:, np.newaxis, np.newaxis])
    weight=(alpha_num/(alpha_den+1e-8))*feature_gradient
    weight = np.mean(weight, axis=(1,2))
    cam=feature_map * weight[:, np.newaxis, np.newaxis]  # [C,H,W]
    cam = np.sum(cam, axis=0)  # [H,W]
    
    cam = cv2.resize(cam, size_sample)
    cam = np.maximum(cam, 0)  # ReLU
    cam=cam-cam.min()
    cam = cam / cam.max()
    hook1.remove()
    hook2.remove()
    
    
    show_critical_semantic_area(cam,img_numpy)
    return cam

In [ ]:
def Class_Saliency_Map(img_numpy,img_tensor,target_label,model):
    img_tensor.requires_grad = True    
    model_output =model(img_tensor)
    model.zero_grad()
    pred_class = model_output.argmax().item()
    
    preds=Func.softmax(model_output,dim=1)
    probs,idx=preds.data.squeeze().sort(0,True)
    probs=probs.data.cpu().numpy()
    idx=idx.data.cpu().numpy()
    i=np.where(idx==target_label)[0]  #index of target class
    #print('{:.3f}->{}:{}\n---------------'.format(probs[i][0],idx[i][0],class_label[idx[i][0]]))

    grad_target_map = torch.zeros(model_output.shape, dtype=torch.float)
    grad_target_map[0][target_label] = 1
 
    model_output.backward(grad_target_map.to(device))
    bp_m =img_tensor.grad.data[0].cpu().data.numpy().squeeze()
    bp_m = np.sum(bp_m, axis=0)  # [H,W]
    bp_m = np.maximum(bp_m, 0)  # ReLU
    bp_m=bp_m-bp_m.min()
    bp_m = bp_m / bp_m.max() #min-max normalization  
    
    show_critical_semantic_area(bp_m,img_numpy)
    return bp_m

In [ ]:
def Guided_Backpropagation(img_numpy,img_tensor,target_label,model):
    
    image_reconstruction=[]
    def first_layer_hook_fn(module, grad_in, grad_out):
        image_reconstruction.append(grad_in[0])  #the first conv layer

    activation_maps=[]
    def forward_hook_fn(module, input, output):
        activation_maps.append(output)

    def backward_hook_fn(module, grad_in, grad_out):
        grad = activation_maps.pop() 
        grad[grad > 0] = 1      
        positive_grad_out = torch.clamp(grad_out[0], min=0.0)   
        new_grad_in = positive_grad_out * grad
        return (new_grad_in,)

        
    for name, module in model.named_modules():
        if isinstance(module, nn.ReLU):
            hook_1=module.register_forward_hook(forward_hook_fn)
            hook_2=module.register_backward_hook(backward_hook_fn)
            
            hook_1.remove()
            hook_2.remove()

    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            first_layer = module
            hook_3=first_layer.register_backward_hook(first_layer_hook_fn)
            break
            
            hook_3.remove()

    model_output = model(img_tensor)
    model.zero_grad()
    pred_class = model_output.argmax().item()
    
    preds=Func.softmax(model_output,dim=1)
    probs,idx=preds.data.squeeze().sort(0,True)
    probs=probs.data.cpu().numpy()
    idx=idx.data.cpu().numpy()
    i=np.where(idx==target_label)[0]  
 
    grad_target_map = torch.zeros(model_output.shape, dtype=torch.float)
    grad_target_map[0][target_label] = 1

        
    model_output.backward(grad_target_map.to(device))
    gbp_m = image_reconstruction[0].data[0].cpu().data.numpy().squeeze()
    gbp_m = np.sum(gbp_m, axis=0)  
    gbp_m = np.maximum(gbp_m, 0)  
    gbp_m=gbp_m-gbp_m.min()
    gbp_m = gbp_m / gbp_m.max()
    
    
    show_critical_semantic_area(gbp_m,img_numpy)
    return gbp_m

In [ ]:
def Semantic_Fusion_Algorithm(image_path,target_label,model): 
    img_numpy,tensor_img=input_image(image_path)
    img_tensor=Variable(torch.from_numpy(tensor_img).to(device).float())
    original_label=np.argmax(model(img_tensor).data.cpu().numpy())    
    #generate grad-cam++
    grad_cam_plus_o1=Grad_CAM_Plus_Plus(img_numpy,img_tensor,original_label,model)
    grad_cam_plus_t1=Grad_CAM_Plus_Plus(img_numpy,img_tensor,target_label,model)    
    #generate saliency_map
    saliency_m_o2=Class_Saliency_Map(img_numpy,img_tensor,original_label,model)
    saliency_m_t2=Class_Saliency_Map(img_numpy,img_tensor,target_label,model)
    #generate guided_bp
    guided_bp_m_o3=Guided_Backpropagation(img_numpy,img_tensor,original_label,model) 
    guided_bp_m_t3=Guided_Backpropagation(img_numpy,img_tensor,target_label,model)
    

    SemanticR_o3=np.where(guided_bp_m_o3>0.1,guided_bp_m_o3,0)   
    SemanticR_o2=np.where(saliency_m_o2>0.1,saliency_m_o2,0)  
    SemanticR_o1=np.where(grad_cam_plus_o1>0.8,grad_cam_plus_o1,0)   

    SemanticR_o=SemanticR_o1+SemanticR_o2+SemanticR_o3
    SemanticR_o=SemanticR_o-SemanticR_o.min()
    SemanticR_o=SemanticR_o/SemanticR_o.max()
    show_critical_semantic_area(SemanticR_o,img_numpy)



    SemanticR_t3=np.where(guided_bp_m_t3>0.2,guided_bp_m_t3,0)  
    SemanticR_t2=np.where(saliency_m_t2>0.2,saliency_m_t2,0)  
    SemanticR_t1=np.where(grad_cam_plus_t1>0.5,grad_cam_plus_t1,0)  

    SemanticR_t=SemanticR_t1+SemanticR_t2+SemanticR_t3
    SemanticR_t=SemanticR_t-SemanticR_t.min()
    SemanticR_t=SemanticR_t/SemanticR_t.max()
    semantic_area_show(SemanticR_t,img_numpy)


    SemanticR=SemanticR_o+SemanticR_t-SemanticR_o*SemanticR_t*2.5
    SemanticR=SemanticR-SemanticR.min()
    SemanticR=SemanticR/SemanticR.max()
    show_critical_semantic_area(SemanticR,img_numpy)

    Semantic_mask=np.zeros_like(img_tensor.data.cpu().numpy())
    Semantic_mask[:,:2,np.where(SemanticR>0)]=1
    return  Semantic_mask,img_numpy, SemanticR

In [ ]:
def Adapted_Adam_BFGS_Algorithm(model,img,orig_label,target_label,Semantic_mask,EPOCH,PerRge):
    target=Variable(torch.from_numpy(np.eye(1000)[target_label]).to(device).float()) #one-hot code
    targetlabel=Variable(torch.Tensor([float(target_label)]).to(device).long())

    theta=0.2
    #define the bound
    max_=3.0
    min_= -3.0 
    imgo=img.data.cpu().numpy().copy()

    img_max= imgo+PerRge
    img_min=imgo-PerRge
    
    m=np.zeros_like(img.data.cpu().numpy())
    v=np.zeros_like(img.data.cpu().numpy())    
    D=np.identity(img.data.cpu().numpy().shape[2])
    beta1=0.9
    beta2=0.999
    lr=0.1
    eps=1e-08
    k=10       
    mask = np.ones_like(img.data.cpu().numpy())  
    

    for epoch in range(EPOCH):
                   
        output=model(img.to(torch.float32))
        label=np.argmax(output.data.cpu().numpy())
        zero_gradients(img)
        
        real=torch.max(output*target)
        other=torch.max((1-target)*output)
        loss=other-real+k
        loss=torch.clamp(loss,min=0)
        loss.backward(retain_graph=True)
        
  
        if label ==target_label: 
            
            print("Iter={} label={}->{} loss={:.6f}\n-------------------------------".format(iter, orig_label,label,loss))
            print("Attack successfully!")
            break            
       

        gradient=img.grad.data.cpu().numpy().copy()
        gradient_pre=gradient
        
        gradient = gradient * mask*Semantic_mask
        idx = np.argmax(gradient)        
        idx = np.unravel_index(idx, mask.shape)        
        if epoch%10==0:
            print(f"Epoch {epoch}")
            print(idx)
            print("label:{}->{} loss={:.6f}\n-------------------------------".format(orig_label,label,loss))
            
        m=beta1*m+(1-beta1)*gradient
        v=beta2*v+(1-beta2)*(gradient**2)
        mm=m/(1-beta1**(epoch+1))
        vv=v/(1-beta2**(epoch+1))
        #mask transformation
        gradient=lr*np.matmul(D,(mm/(np.sqrt(vv)+eps)))

        img_pre=img     
        img.data[idx]=img.data[idx]-gradient[idx] #* theta * (max_ - min_) 
        
        output=model(img)
        real=torch.max(output*target)
        other=torch.max((1-target)*output)
        loss=other-real+k
        loss=torch.clamp(loss,min=0)
        loss.backward(retain_graph=True)    
        gradient=img.grad.data.cpu().numpy().copy()
        
       
        sk=img.data.cpu().numpy()-img_pre.data.cpu().numpy()
        yk=gradient-gradient_pre
        
        rk=1/(np.matmul(np.transpose(yk,(0,1,3,2)),sk)+1e-10)
        term1=rk*np.matmul(sk,np.transpose(yk,(0,1,3,2)))
        term2=rk*np.matmul(yk,np.transpose(sk,(0,1,3,2)))
        I=np.identity(img.data.cpu().numpy().shape[2])
        term3=np.matmul(I-term1,D)
        term4=np.matmul(term3,I-term2)
        term5=rk*np.matmul(sk,np.transpose(sk,(0,1,3,2)))
        D=term4+term5  
            
        if (img.data[idx]<=img_min[idx]) or (img.data[idx]>=img_max[idx]):
            print("idx={} over {:.6f},{:.6f},{:.6f}".format(idx,img.data[idx],img_min[idx],img_max[idx]))
            mask[idx]=0
            img.data[idx]=np.clip(img.data[idx].cpu(), img_min[idx], img_max[idx]) 
        
        if (img.data[idx]<=min_) or (img.data[idx]>=max_):
            print("idx={} over {:.6f},{:.6f},{:.6f".format(idx,img.data[idx].cpu(),min_,max_))
            mask[idx]=0
            img.data[idx]=np.clip(img.data[idx], min_, max_)
    return img,label,imgo,epoch+1

In [ ]:
def Targeted_Attack_SFGA(img,model, target_label,Semantic_mask,EPOCH,PerRge):
    orig_label=np.argmax(model(img).data.cpu().numpy())
    print("orig_label={}:{}".format(orig_label,class_label[orig_label]))    
    img.requires_grad = True
    for param in model.parameters():
        param.requires_grad = False
   
    
    adv_img,label,imgo,epoch=Adapted_Adam_BFGS_Algorithm(model,img,orig_label,target_label,Semantic_mask,EPOCH,PerRge)  #   gradient_adam  #gradient_backpropagation  
        
    preds_=Func.softmax(model(adv_img),dim=1)
    probs_,idx_=preds_.data.squeeze().sort(0,True)
    probs_=probs_.data.cpu().numpy()
    idx_=idx_.data.cpu().numpy()
    for k_ in range(10):
        print('{}, {},  {:.5f}, {}'.format(k_+1, idx_[k_], probs_[k_], class_label[idx_[k_]]))   
    
    
    adv= adv_img.data.cpu().numpy()[0]
    adv = adv.transpose(1, 2, 0)

    adv = (adv * std) + mean
    adv = adv * 256.0
    adv = np.clip(adv, 0, 255).astype(np.uint8)
    
    
    
    print("original {}:{}".format(orig_label,class_label[orig_label]))
    print("current {}:{}".format(label,class_label[label]))
    return orig_label,adv,label,epoch